# Deep Learning Mini-Challenge 1: Image Classification with Neuronal Networks

**Task description:** The goal of this mini-challenge is the implementation, optimization and evaluation of a neural network model for the classification of an image dataset. The focus is on the training and evaluation of the models. The evaluation is limited to Multi layer Perceptrons (MLP's) and Convolutional neural Networks (CNN's) in combination with different optimization and regularization methods.

**Description of the dataset:** For this mini challenge we use the `CIFAR10` dataset, which is a sub-set of the `Tiny images` dataset. The dataset is named after the **C**anadian **I**nstitute **F**or **A**dvanced **R**esearch. It is an image dataset and contains some 60 000 labelled images that belong to one of 10 classes. The images have a resolution of 32x32 pixels and are in RGB format. 


### Load dependencies

In [363]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from multiprocessing import cpu_count

import wandb

import copy
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime

os.environ["WANDB_SILENT"] = "True"

### Device configuration

Check for allocated devices that can be used for training of the Models.

In [364]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Weights and Biases init configuration

In [365]:
wandb.login()

True

In [366]:
import pandas as pd 
import wandb

def read_wandb_logs():
    '''reads wandb logs of all runs'''
    api = wandb.Api()
    entity, project = "simonluder", "del-mc1"  # set to your entity and project 
    runs = api.runs(entity + "/" + project) 

    summary_list, config_list, name_list, history_list = [], [], [], []
    for i, run in enumerate(runs): 
        if i % 20 == 0:
            print("Reading model:", i)

        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .history creates a history dataframe of all logged metrics
        history_list.append(run.history())

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
             if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    # combine into single dataframes
    df_history_summary = pd.DataFrame()
    df_config_summary = pd.DataFrame()
    for i, name in enumerate(name_list):
        history_list[i]["name"] = name
        df_history_summary = pd.concat([df_history_summary, history_list[i]])
        config_list[i]["name"] = name
        df_config_summary = df_config_summary.append(config_list[i], ignore_index=True)
    df_history_summary = df_history_summary.reset_index(drop=True)

    return df_config_summary, df_history_summary

Reading model: 0
Reading model: 20
Reading model: 40
Reading model: 60
Reading model: 80
Reading model: 100


## Load Dataset

Pytorchs torchvision library offers access to a variety of datasets, including CIFAR10. In a first step, the dataset is downloaded and stored in the data folder of our repository. In the future, our dataset will be automatically accessed from there. 

After that, the dataset gets loaded into the notebook without any transformation to review the data and conduct some analysis. The data is loaded using pytorch's `DataLoader` method. This allows for relatively simple pre-sampling and loading of the data, with the possibility of custom customizing loading order and automatic batching. For this notebook we use a batch size of 32 samples per batch.

In [ ]:
def define_dataset(normalize=False, path=None):
    '''Defines the train and testset
    Args:
        normalize (bool): if true normalizes the image
    Returns:
        train_set (torchvision dataset)
        test_set (torchvision dataset)
    '''
    if normalize:
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    else:
        transform = transforms.Compose([transforms.ToTensor(),])

    if not os.path.exists(path):
        os.makedirs(path)

    train_set = torchvision.datasets.CIFAR10(
        root=path, 
        train=True, 
        download=True, 
        transform=transform)

    test_set = torchvision.datasets.CIFAR10(
        root=path, 
        train=False, 
        download=True, 
        transform=transform)

    return train_set, test_set


def define_dataloader(train_set, test_set, batch_size):
    '''Defines the dataloader for the train and test set
    Args:
        train_set (torchvision dataset):
        test_set (torchvision dataset):
        batch_size (int): defines the number of samples per batch
    Returns:
        train_loader
        test_loader
    '''
    train_loader = torch.utils.data.DataLoader(
        train_set, 
        batch_size=batch_size, 
        shuffle=True,
        )

    test_loader = torch.utils.data.DataLoader(
        test_set, 
        batch_size=batch_size, 
        shuffle=False,
        )

    return train_loader, test_loader


# load dataset without normalization
batch_size = 32
train_set, test_set =  define_dataset(normalize=False, path='./../../data')
train_loader, test_loader = define_dataloader(train_set, test_set, batch_size)

### Define class labels

The CIFAR10 dataset contains ten different classes. In the dataset itself, these are represented by the numerical values 0 to 9. In order to be able to check the individual samples better, we create a dictionary that creates a mapping between the numerical class labels and the real class names.

In [ ]:
classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
classes_map = dict(zip(range(len(classes)), classes))
classes_map

## Explorative data analysis

### Class balance in training and test set

We now want to check the frequencies of the individual class labels in the training and test sets.

In [ ]:
# retrieve train label class count
train_label_count = list()
for samples, labels in train_loader:
    train_label_count.extend(labels.numpy())
train_label_count = {classes_map[i]: train_label_count.count(i) for i in set(train_label_count)}

plt.figure(figsize=(10,5))
plt.bar(list(train_label_count.keys()), list(train_label_count.values()))
plt.title("Nr of labels per class in train set")
plt.ylabel("count")
plt.show()

**Description:** As can be seen in the visualisation above, the training set has balanced class labels. There are 5000 labels per class for all ten classes which results in 50'000 labels total within the trainig set. Next we want to examine the test set.

In [ ]:
# retrieve test label class count
test_label_count = list()
for samples, labels in test_loader:
    test_label_count.extend(labels.numpy())
test_label_count = {classes_map[i]: test_label_count.count(i) for i in set(test_label_count)}

plt.figure(figsize=(10,5))
plt.bar(list(test_label_count.keys()), list(test_label_count.values()))
plt.title("Nr of labels per class in test set")
plt.ylabel("count")
plt.show()

**Description:** The test set also has balanced class labels. There are 1000 samples per class in the test set which means in total, our test set has 10,000 samples. Knowing the size of the train and test set are able to retrieve calculate the split ratio in the CIFAR10 dataset. The resulting split ratio is therefore 5 to 1 between the train and test set.

### Select sample batch of images

In [ ]:
example_batch = iter(train_loader)
samples, labels = example_batch.next()
print("Sample Dimensions:", samples.shape, "\nLabel Dimension:", labels.shape)

**Description:** We now see the dimensions of a selected batch from the training set. The sample dimension is 32, 3, 32, 32. These dimensions are composed as follows:
**32** samples,
**3** colour dimensions (RGB),
**32** pixels image height,
**32** pixels image width.
The label dimension is simply 32 for the labels of the **32** samples.

### Visualization of a single batch

In [ ]:
m = 8
n = batch_size // m

fig, axes = plt.subplots(m, n, figsize=(16,16))
for i, image in enumerate(range(batch_size)):
    axes[i//n, i%n].imshow(samples[i].permute(1, 2, 0))
    axes[i//n, i%n].title.set_text("Class: {}".format(classes[labels[i]]))
    axes[i//n, i%n].set_xticks([])
    axes[i//n, i%n].set_yticks([])


plt.suptitle("Visualizing images from training set")
plt.tight_layout()
plt.show()

**Description:** Above we see our randomly drawn training batch. The respective class labels are present in the title. As we can see, the images are not always easy to recognise, even for a human being, due to the low resolution. The biggest advantage of the minimal resolution is the reduction of the features that our model has to take into account, which means that we can use simpler models and correspondingly need less training time.

## Define the target metrics 
In this notebook, a set of metrics are used for the evaluation. 

Since we want to solve a multiclass classification task, the hyperparameter optimization is done primarily by using the cross entropy loss and the balanced accuracy. The score for a perfect prediction would be zero. The initial score of a modell should therefore be higher and start to converge to a lower value. For more information on pytorchs CrossEntropyLoss see: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

Because the dataset has the same proportion of labels within each of its classes, the accuracy equals the balanced accuracy. The accuracy varies in theory between 0 which is the worst possible and 1 which is a perfect prediction. In reality, the accuracy score of our models should start at around 0.1 since we have 10 classes, and it is what could be expected on random prediction of untrained models. 


## Define Models
The individual model classes are initialised here.

In [ ]:
class MLPNet_2_layer(nn.Module):
    '''Multi Layer Perceptron Net class with 2 layers'''
    def __init__(self, fc_hidden_1, fc_hidden_2=None, fc_hidden_3=None, activation=F.relu, drop_out=0):
        super(MLPNet_2_layer, self).__init__()
        self.fc1 = nn.Linear(in_features=32*32*3, out_features=fc_hidden_1)
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation # activation function
    
    def forward(self, x):
        batch_size  = x.shape[0]
        x = x.view(batch_size, -1)
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class MLPNet_3_layer(nn.Module):
    '''Multi Layer Perceptron Net class with 3 layers'''
    def __init__(self, fc_hidden_1, fc_hidden_2, fc_hidden_3=None, activation=F.relu, drop_out=0):
        super(MLPNet_3_layer, self).__init__()
        self.fc1 = nn.Linear(in_features=32*32*3, out_features=fc_hidden_1)
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation # activation function
    
    def forward(self, x):
        batch_size  = x.shape[0]
        x = x.view(batch_size, -1)
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

class MLPNet_4_layer(nn.Module):
    '''Multi Layer Perceptron Net class with 4 layers'''
    def __init__(self, fc_hidden_1, fc_hidden_2, fc_hidden_3, activation=F.relu, drop_out=0):
        super(MLPNet_4_layer, self).__init__()
        self.fc1 = nn.Linear(in_features=32*32*3, out_features=fc_hidden_1)
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=fc_hidden_3)
        self.fc4 = nn.Linear(in_features=fc_hidden_3, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation # activation function
    
    def forward(self, x):
        batch_size  = x.shape[0]
        x = x.view(batch_size, -1)
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.activation(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x
    
class MLPNet_6_layer(nn.Module):
    '''Multi Layer Perceptron Net class with 4 layers'''
    def __init__(self, fc_hidden_1, fc_hidden_2, fc_hidden_3, fc_hidden_4, fc_hidden_5, activation=F.relu, drop_out=0):
        super(MLPNet_6_layer, self).__init__()
        self.fc1 = nn.Linear(in_features=32*32*3, out_features=fc_hidden_1)
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=fc_hidden_3)
        self.fc4 = nn.Linear(in_features=fc_hidden_3, out_features=fc_hidden_4)
        self.fc5 = nn.Linear(in_features=fc_hidden_4, out_features=fc_hidden_5)
        self.fc6 = nn.Linear(in_features=fc_hidden_5, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation # activation function
    
    def forward(self, x):
        batch_size  = x.shape[0]
        x = x.view(batch_size, -1)
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.activation(self.fc3(x))
        x = self.dropout(x)
        x = self.activation(self.fc4(x))
        x = self.dropout(x)
        x = self.activation(self.fc5(x))
        x = self.dropout(x)
        x = self.fc6(x)
        return x

In [ ]:
# load dataset without normalization
train_set, test_set =  define_dataset(normalize=True, path='./../../data')
train_loader, test_loader = define_dataloader(train_set, test_set, batch_size)

## Define Model Trainer
The `ModelTrainer` class trains the network and logs the metrics that are calculated during training. The class gets as an input the model that needs to be trained, a train-, and test-loader as well as the number of epochs to train, an optimizer as well as a criterion that needs to be optimized.

To speed the evaluation up, the training per model is stopped after the specified number of epochs is reached or if the top test accuracy did not increase for more than 25 epochs.

In [ ]:
class ModelTrainer:
    
    def __init__(self, model, train_loader, test_loader, num_epochs, criterion, optimizer):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.num_epochs = num_epochs
        self.criterion = criterion
        self.optimizer = optimizer
        self.logger = {
            "train":dict(),
            "test":dict()
        }
        self.best_log = dict()
        self.best_model = None
        
        
    def wandb_log(self, log_dict, e):
        wandb.log(log_dict, step=e)


    def calculate_accuracy(self, pred, true):
        '''Calculates the accuracy for the prediction
        Args: 
            pred: predicted labels
            true: true labels
        Returns:
            accuracy as float
        '''
        return (pred.argmax(1) == true).type(torch.float).sum().item()
    

    def train_loop(self, dataloader, verbose):
        '''
        Loop for a sigle training epoch.
        Args:
            dataloader: pytorch dataloader
        '''
        epoch_accuracy = 0
        epoch_loss = 0
        num_batches = len(dataloader)

        for i, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)
            # Forward propagation
            pred = self.model(images)
            loss = self.criterion(pred, labels)
            
            # Backward propagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # calculate metrics
            epoch_loss += loss.item()
            epoch_accuracy += self.calculate_accuracy(pred, labels)

        epoch_loss /= num_batches
        epoch_accuracy /= len(dataloader.dataset)
        if verbose:
            print(f"train loss:{epoch_loss}, train accuracy:{epoch_accuracy}")
        return {"train loss":epoch_loss, "train accuracy":epoch_accuracy}


    def test_loop(self, dataloader, label="train"):
        epoch_loss = 0
        epoch_accuracy = 0
        num_batches = len(dataloader)

        with torch.no_grad():
            for i, (images, labels) in enumerate(dataloader):
                images = images.to(device)
                labels = labels.to(device)

                # Forward propagation
                pred = self.model(images)

                # calculate metrics
                epoch_loss += self.criterion(pred, labels).item()
                epoch_accuracy += self.calculate_accuracy(pred, labels)
        epoch_loss /= num_batches
        epoch_accuracy /= len(dataloader.dataset)
        return {"{} loss".format(label): epoch_loss, "{} accuracy".format(label): epoch_accuracy}


    def train(self, verbose=False, include_test_scores=False, early_stopping=False):
        # calculate initial model preformance
        self.logger["train"][0] = self.test_loop(self.train_loader, label="train")
        self.wandb_log(self.logger["train"][0], 0)
        if include_test_scores:
            self.logger["test"][0] = self.test_loop(self.test_loader, label="test")
            self.wandb_log(self.logger["test"][0], 0)
            
            # save init as best model
            self.best_log = self.logger["test"][0]
            self.best_model = copy.deepcopy(self.model)
            self.logger["test"][0]["top test accuracy"] = self.logger["test"][0]["test accuracy"]
            self.wandb_log({"top test accuracy":self.best_log["test accuracy"]}, 0)
            

        for e in range(self.num_epochs):
            if verbose:
                print(f"Epoch: {e+1} ------------------------------")
            
            # train model
            self.logger["train"][e+1] = self.train_loop(self.train_loader, verbose)
            self.wandb_log(self.logger["train"][e+1], e+1)

            # calculate model preformance
            if include_test_scores:
                self.logger["test"][e+1] = self.test_loop(self.test_loader, label="test")
                self.wandb_log(self.logger["test"][e+1], e+1)

                # secure model with best test score
                if self.best_log["test accuracy"] < self.logger["test"][e+1]["test accuracy"]:
                    self.best_log = self.logger["test"][e+1]
                    self.best_model = copy.deepcopy(self.model)
                self.logger["test"][e+1]["top test accuracy"] = self.best_log["test accuracy"]
                self.wandb_log({"top test accuracy":self.best_log["test accuracy"]}, e+1)
                
                # stop if test accuracy stops increasing for 25 epochs
                if (e >= 25) and early_stopping:
                    if self.best_log["test accuracy"] == self.logger["test"][e-24]["top test accuracy"]:
                        break



In [ ]:
# num_epochs = 3
# model = ConvNet_5(
#             conv1_channels=120,
#             conv2_channels=120,
#             conv3_channels=120,
#             conv4_channels=120,
#             conv5_channels=120).to(device) # model selection
# learning_rate = 0.01        # learning rate for optimizer
# l2_weight_decay = 0.0       # l2 regularization for optimizer

# trainer = ModelTrainer(
#     model=model, 
#     train_loader=train_loader, 
#     test_loader=test_loader, 
#     num_epochs=num_epochs,
#     criterion=nn.CrossEntropyLoss(),
#     optimizer=torch.optim.SGD(
#         model.parameters(), 
#         lr=learning_rate,
#         weight_decay = l2_weight_decay
#         )
#     )

# trainer.train(verbose=True, include_test_scores=True)
# trainer.logger

## Evaluate Model Size

### MLP architecture

In a first step we want to train a simple MLP with variable number of layers and nodes to see how well it is able to learn the trainigset and to use the trained network for evaluation of the classifications on the testset. The Multilayer Perceptron recieves the input data as a one-dimensional vector per image or as a two-dimensional matrix for multiple images. 

The structure of the network contains **1**, **2**, **3**, or **5** hidden layers. The nummber of nodes per layer varies between **32**, **64**, **128** and **256** nodes. We train four models for each structural combination with a varying learning rate of **0.02**, **0.05**, **0.1** and **0.2**. This results in **4 * 4 * 4 = 64** different trained mlp's.

### CNN architecture

The architecture of a Convolutional Neural Network extends the existing structure of our MLP's with Convolutional Layers, followed by a Pooling Layer. The convolutional layer receives as input our 2 dimensional images and the three colour channels (RGB) as a 3d matrix. Then a filter kernel is iterated over the pixels of the matrix window by window. The output for a single pixel is calculated as the scalar product of the iterated window and the filter kernel. The output dimension of the Concolutional Layer can be calculated using the formula below.

</br>

$$H_{out} = \left\lfloor\frac{H_{in} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1\right\rfloor$$

$$W_{out} = \left\lfloor\frac{W_{in} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1\right\rfloor$$

where

- $W_{in}$: Width of input
- $H_{in}$: Height of input

The subsequent pooling layer is used to filter out excess information. Here, again, a window is iterated over the output of the convolutional layer and a pooling action is performed. The most common method is max pooling, but average pooling is also used for deeper neural networks. This reduces the features of our matrices, while it usually does not reduce the accuracy of our network. Due to the feature reduction, our network is not only faster in computation and training, but it is also possible to train deeper networks. Furthermore, this method prevents overfitting, since the individual features are viewed in a generalised manner within the local iterated window. 


The structure of the cnn contains **2**, **3**, or **5** convolutional layers followed by a pooling layer. The nummber of filter kernel per convolutional layer varies between **32**, **64**, **128** and **256** filters. And like before we train four models for each structural combination with a varying learning rate of **0.02**, **0.05**, **0.1** and **0.2**. This results in **3 * 4 * 4 = 48** different trained cnn's.

In [ ]:
class ConvNet_2(nn.Module):
    '''A convolutional neural network with 2 convolutional layers'''
    def __init__(self, activation=F.relu, kernel_size=5, pool_size=2, conv1_channels = 10, conv2_channels=8, fc_hidden_1=120, fc_hidden_2=84, drop_out=0):
        '''Convolutional Net class'''
        super(ConvNet_2, self).__init__()   
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=conv1_channels, kernel_size=kernel_size) 
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
        self.conv2 = nn.Conv2d(in_channels=conv1_channels, out_channels=conv2_channels, kernel_size=kernel_size) 
        self.fc1 = nn.Linear(in_features=conv2_channels*5*5, out_features=fc_hidden_1)   
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation
        self.conv2_channels = conv2_channels

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = x.view(-1, self.conv2_channels*5*5) # flatten tensor
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

class ConvNet_3(nn.Module):
    def __init__(self, activation=F.relu, kernel_size=5, pool_size=2, padding=3, conv1_channels = 10, conv2_channels=8, conv3_channels=8, fc_hidden_1=120, fc_hidden_2=84, drop_out=0):
        '''A convolutional neural network with 3 convolutional layers'''
        super(ConvNet_3, self).__init__()   
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=conv1_channels, kernel_size=kernel_size, padding=padding) 
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
        self.conv2 = nn.Conv2d(in_channels=conv1_channels, out_channels=conv2_channels, kernel_size=kernel_size, padding=padding) 
        self.conv3 = nn.Conv2d(in_channels=conv2_channels, out_channels=conv3_channels, kernel_size=kernel_size, padding=padding)
        self.fc1 = nn.Linear(in_features=conv3_channels*5*5, out_features=fc_hidden_1)   
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation
        self.conv3_channels = conv3_channels

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.pool(self.activation(self.conv3(x)))
        x = x.view(-1, self.conv3_channels*5*5) # flatten tensor
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

    
class ConvNet_5(nn.Module):
    def __init__(self, activation=F.relu, kernel_size=5, pool_size=2, padding=3, conv1_channels = 16, conv2_channels=16, conv3_channels=16, conv4_channels=16, conv5_channels=16, fc_hidden_1=120, fc_hidden_2=84, drop_out=0):
        '''A convolutional neural network with 3 convolutional layers'''
        super(ConvNet_5, self).__init__()   
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=conv1_channels, kernel_size=kernel_size, padding=padding) 
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
        self.conv2 = nn.Conv2d(in_channels=conv1_channels, out_channels=conv2_channels, kernel_size=kernel_size, padding=padding) 
        self.conv3 = nn.Conv2d(in_channels=conv2_channels, out_channels=conv3_channels, kernel_size=kernel_size, padding=padding)
        self.conv4 = nn.Conv2d(in_channels=conv3_channels, out_channels=conv4_channels, kernel_size=kernel_size, padding=padding)
        self.conv5 = nn.Conv2d(in_channels=conv4_channels, out_channels=conv5_channels, kernel_size=kernel_size, padding=padding)
        self.fc1 = nn.Linear(in_features=conv5_channels*2*2, out_features=fc_hidden_1)   
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation
        self.conv5_channels = conv5_channels

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.pool(self.activation(self.conv3(x)))
        x = self.pool(self.activation(self.conv4(x)))
        x = self.pool(self.activation(self.conv5(x)))
        x = x.view(-1, self.conv5_channels*2*2) # flatten tensor
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [ ]:
ConvNet_5()

## Train models

As a last setup, a function is created that contains the initialisation of our model, the ModelTrainer, the subsequent training, logging and saving of the best models. In itself, this function is not really necessary, but it allows us to prepare individual packages of constellations that we can then simply recall afterwards, which saves the repetition of code.

In [ ]:
def save_model(path, model, name):
        '''
        Saves the model as state dict.
        Args:
            path (str): path where to save the model
        '''
        filename = "/{}.pt".format(name)
        if not os.path.exists(path):
            os.makedirs(path)
        torch.save(model, path + filename)
        

def train_mlp_2(config=None):
    '''Trains and logs a multilayer perceptron'''
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        
        # define model
        model = MLPNet_2_layer(
            fc_hidden_1=config["hidden_layers"][0],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"]))
        
        #train model
        trainer.train(verbose=False, include_test_scores=True, early_stopping=True)
        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)


def train_mlp_3(config=None):
    '''Trains and logs a multilayer perceptron'''
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        
        # define model
        model = MLPNet_3_layer(
            fc_hidden_1=config["hidden_layers"][0],
            fc_hidden_2=config["hidden_layers"][1],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"]))
        
        #train model
        trainer.train(verbose=False, include_test_scores=True, early_stopping=True)
        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)


def train_mlp_4(config=None):
    '''Trains and logs a multilayer perceptron'''
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        
        # define model
        model = MLPNet_4_layer(
            fc_hidden_1=config["hidden_layers"][0],
            fc_hidden_2=config["hidden_layers"][1],
            fc_hidden_3=config["hidden_layers"][2],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"]))
        
        #train model
        trainer.train(verbose=False, include_test_scores=True, early_stopping=True)
        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
            
def train_mlp_6(config=None):
    '''Trains and logs a multilayer perceptron'''
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        
        # define model
        model = MLPNet_6_layer(
            fc_hidden_1=config["hidden_layers"][0],
            fc_hidden_2=config["hidden_layers"][1],
            fc_hidden_3=config["hidden_layers"][2],
            fc_hidden_4=config["hidden_layers"][3],
            fc_hidden_5=config["hidden_layers"][4],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"]))
        
        #train model
        trainer.train(verbose=False, include_test_scores=True, early_stopping=True)
        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
            
def train_cnn_2(config=None):
    '''Trains and logs a convolutional neuronal network'''
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        
        # define model
        model = ConvNet_2(
            kernel_size=config["kernel_size"],
            pool_size=config["pool_size"],
            conv1_channels=config["conv_layers"][0],
            conv2_channels=config["conv_layers"][1],
            fc_hidden_1=config["hidden_layers"][0],
            fc_hidden_2=config["hidden_layers"][1],
            drop_out=config["drop_out"],
            ).to(device) 
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"]))
        
        #train model
        trainer.train(verbose=False, include_test_scores=True, early_stopping=True)
        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
            
def train_cnn_3(config=None):
    '''Trains and logs a convolutional neuronal network'''
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        
        # define model
        model = ConvNet_3(
            kernel_size=config["kernel_size"],
            pool_size=config["pool_size"],
            conv1_channels=config["conv_layers"][0],
            conv2_channels=config["conv_layers"][1],
            conv3_channels=config["conv_layers"][2],
            fc_hidden_1=config["hidden_layers"][0],
            fc_hidden_2=config["hidden_layers"][1],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"]))
        
        #train model
        trainer.train(verbose=False, include_test_scores=True, early_stopping=True)
        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
            
def train_cnn_5(config=None):
    '''Trains and logs a convolutional neuronal network'''
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        
        # define model
        model = ConvNet_5(
            kernel_size=config["kernel_size"],
            pool_size=config["pool_size"],
            conv1_channels=config["conv_layers"][0],
            conv2_channels=config["conv_layers"][1],
            conv3_channels=config["conv_layers"][2],
            conv4_channels=config["conv_layers"][3],
            conv5_channels=config["conv_layers"][4],
            fc_hidden_1=config["hidden_layers"][0],
            fc_hidden_2=config["hidden_layers"][1],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"]))
        
        #train model
        trainer.train(verbose=False, include_test_scores=True, early_stopping=True)
        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)

## Model Evaluation

In this section, different model constelations are examined for their performance. The evaluation is done by using Baesiean Search. The optimization is done with respect to the accuracy.

### Training of MLP's with SGD (without REG, without BN)

In [ ]:
num_epochs = 100
min_acc = 0.5
create_sweep = False #create new sweeps

# create initial sweep configuration
mlp_sweep_configuration = {
    "name": "MLP_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "grid",
    "parameters": {
        "variant": {
            "value": "MLP_SGD"
        },
        "num_hidden": {
            "value": 1
        },
        "hidden_layers": {
            "values":[(32,), (64,), (128,), (256,)]
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.2, 0.1, 0.05, 0.02]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "value": 0
        },
    }
}

# run 2 layer mlp
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del-mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train_mlp_2)

    
# update sweep configuration for 3 layers mlp
mlp_sweep_configuration["parameters"].update({
    "num_hidden": {"value": 2},
    "hidden_layers": {"values":[(32,32), (64,64), (128,128), (256,256)]}})
# run 3 layer mlp
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del-mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train_mlp_3)

    
# update sweep configuration for 4 layers mlp
mlp_sweep_configuration["parameters"].update({
    "num_hidden": {"value": 3},
    "hidden_layers": {"values":[(32,32,32), (64,64,64), (128,128,128), (256,256,256)]}})
# run 4 layer mlp
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del-mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train_mlp_4)

    
# update sweep configuration for 6 layers mlp
mlp_sweep_configuration["parameters"].update({
    "num_hidden": {"value": 5},
    "hidden_layers": {"values":[(32,32,32,32,32), (64,64,64,64,64), (128,128,128,128,128), (256,256,256,256,256)]}}) 
# run 6 layer mlp
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del-mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train_mlp_6)

### Training of CNN's with SGD (without REG, without BN)

In [ ]:
num_epochs = 100
min_acc = 0.75
create_sweep = False #create new sweeps

# create initial sweep configuration
cnn_sweep_configuration = {
    "name": "CNN_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "grid",
    "parameters": {
        "variant": {
            "value": "CNN_SGD"
        },
        "num_conv": {
            "value": 2
        },
        "conv_layers": {
            "values": [(32, 32),(64, 64),(128,128),(256,256)]
        },
        "num_hidden": {
            "value": 2
        },
        "hidden_layers": {
            "values": [(128, 64),]
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.2, 0.1, 0.05, 0.02]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "value": 0
        },
        "kernel_size": {
            "value": 5
        },
        "pool_size": {
            "value": 2
        },
    }
}

# run 2 cnn 2 mlp layer
if create_sweep:
    sweep_id = wandb.sweep(cnn_sweep_configuration, project="del-mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train_cnn_2)

    
# update sweep configuration for 3 layers cnn
cnn_sweep_configuration["parameters"].update({
    "num_conv": {"value": 3},
    "conv_layers": {"values":[(32,32,32), (64,64,64), (128,128,128), (256,256,256)]}})
# run 3 cnn 2 mlp layer
if create_sweep:
    sweep_id = wandb.sweep(cnn_sweep_configuration, project="del-mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train_cnn_3)

    
# update sweep configuration for 5 layers cnn
cnn_sweep_configuration["parameters"].update({
    "num_conv": {"value": 5},
    "conv_layers": {"values":[(32,32,32,32,32), (64,64,64,64,64), (128,128,128,128,128), (256,256,256,256,256)]}})
# run 5 cnn 2 mlp layer
if create_sweep:
    sweep_id = wandb.sweep(cnn_sweep_configuration, project="del-mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train_cnn_5)

### Evaluation

In [372]:
df_history_max_test_acc = df_history_summary.loc[df_history_summary.groupby(["name"])["test accuracy"].idxmax()].reset_index(drop=True)
df_history_max_test_acc = pd.merge(df_history_max_test_acc, df_config_summary, left_on="name", right_on="name")

In [375]:
def print_best_models(df, variant=None):
    '''Prints the best model vor a specified variant'''
    # select model variant
    if variant:
        df = df.loc[df["variant"]==variant]
    # select model with best test accuracy
    df = df.loc[df["top test accuracy"]==np.max(df["top test accuracy"])]
    print("Best \033[1m{}\033[0m model reached a test accuracy of \033[1m{}\033[0m at epoch \033[1m{}\033[0m with and test loss of \033[1m{:.3f}\033[0m".format(
        df["variant"].values[0], 
        df["test accuracy"].values[0],
        df["_step"].values[0].astype(int), 
        df["test loss"].values[0]
    ))

print_best_models(df_history_max_test_acc, variant="MLP_SGD")
print_best_models(df_history_max_test_acc, variant="CNN_SGD")

Best MLP_SGD model reached a test accuracy of 0.5485 at epoch 13 with and test loss of 1.370
Best CNN_SGD model reached a test accuracy of 0.8077 at epoch 79 with and test loss of 1.884


**Description:** The best MLP achieved a test accuracy of **0.549**, the best CNN **0.808**. The cnn thus performs significantly better on previously unseen data. It is noticeable that the best mlp has the maximum test accuracy already in epoch **13** while the cnn performed best on the test set in epoch **79**. We want to examine the resulting scores in more detail in our next steps.

#### MLP metrics development

In [426]:
def show_training(df, variant, columns):
    if variant:
        df = df.loc[df["variant"]==variant]
    for col in columns:
        show_training_single_plot(df, variant, col)

    
def show_training_single_plot(df, variant, col):
    '''Visualize training of a single metric'''
    if variant:
        df = df.loc[df["variant"]==variant]
        
    color_dict = {0.2:"green", 0.1:"orange", 0.05:"red", 0.02:"blue"}
    df["color_lr"] = df["learning_rate"].apply(lambda x: color_dict[x])

    fig = go.Figure()
    # Add traces
    for name in set(df["name"]):
        fig.add_trace(go.Scatter(x=df[df["name"]==name]._step, 
                                 y=df[df["name"]==name][col], 
                                 mode='lines',
                                 marker_color= list(set(df[df["name"]==name]["color_lr"]))[0],
                                 name=name))
        
    fig.update_layout(title_text="{} over epochs".format(col), 
                      xaxis_title="epochs",
                      yaxis_title=col,
#                       hovermode="x unified",
                      height=400)

    fig.show()

show_training(df = pd.merge(df_history_summary, df_config_summary, left_on="name", right_on="name"), 
              variant = "MLP_SGD", 
              columns = ["train loss", "test loss", "train accuracy", "test accuracy", "top test accuracy"])

**Description:** We see that the training loss converges relatively stable towards a minimum for all models. The training accuracy is similar, which shows us that our mlp's work correctly in terms of training. If we compare the training and test loss, we see that the test loss reaches a minimum fairly early on. Then the test loss starts to increase again, while the training loss continues to decrease, which indicates an over fitting of the model.

If we compare the test loss with the test accuracy, we see that the test accuracy also reaches the best result around the same time as the test loss. Afterwards, the test accuracy decreases again, but less severely than the test loss. This may be due to the fact that the model causes a sharp increase in the test loss due to the overfitting of the training data set, but the classification with the absolute labels remains approximately the same.

The colors reflect the learning rate of the network. There seems to be a correlation between the learning rate and the effectiveness of the training. After the inspection of the cnn metrics we will therefore investigate the effect of the learning rate on the test accuracy.

#### CNN metrics development

In [427]:
show_training(df = pd.merge(df_history_summary, df_config_summary, left_on="name", right_on="name"), 
              variant = "CNN_SGD", 
              columns = ["train loss", "test loss", "train accuracy", "test accuracy", "top test accuracy"])

**Description:** 



For the CNNs we see a much faster convergence of the loss functions. Here, we also see an overfitting of most models from about the 10th epoch onwards, propably because of the same reasons as with the mlp's.

With the CNN, we can clearly see how the learning rate can influence the training.The learning rate of 0.2 and 0.1 is clearly too high, which can be seen in the explosion of the loss function during training. With the higher learning rates, the optimum for the test accuracy and the test loss is also reached relatively early in the training. For lower learning rates the highest test accuracy score can be reached later in training. For this reason, in the next step we would like to take a closer look at the learning rate per model variant.

#### Influence of learning rate on training

In [428]:
def create_plot_sizes(df):
    fig = px.scatter(
        df, 
        x = df["variant"].astype(str) + df["conv_layers"].astype(str).replace("nan", "") + df["hidden_layers"].astype(str) ,
        y = df["top test accuracy"],
        color = df["learning_rate"],
    )
    
    fig.update_layout(
        title_text='Best test accuracy scores for varying layers learning rates per model',
        yaxis_title="best test accuracy",
        xaxis_title="network variants",
        legend_title="learning rate",
        height=800, 
    )
    fig.update_traces(marker=dict(size=12))
    fig.show()

create_plot_sizes(df_history_max_test_acc)

The visualization shows the best achieved test accuracy according to model variant and learning rate. It can be seen that an increase in the learning rate led to a deterioration in the best test accuracy across all variants. For this reason, I would recommend using a learning rate of **0.05** or less on our data set for training with the mlp as well as the cnn.

The visualization also shows the difference between the generalization ability of the MLP and the CNN variants. Finally we want to investigate the influence of the number of layers and the number of nodes per layer. 

#### MLP influence of  layer constellations

In [373]:
def compare_mlp_sizes(df, variant):
    if variant:
        df = df.loc[df["variant"]==variant]
        
    df["hidden_layer_size"] = df["hidden_layers"].apply(lambda x: x[0])
    df = df[["num_hidden", "hidden_layer_size", "test accuracy"]].groupby(["num_hidden", "hidden_layer_size"]).max().reset_index()
    df = df.pivot(index='num_hidden', columns='hidden_layer_size', values='test accuracy')
   
    fig = go.Figure(data=go.Heatmap(
        z = df.values,
        x = df.columns.astype(str),
        y = df.index.astype(str),
        texttemplate="%{z}",
        textfont={"size":20}
        ))
    
    fig.update_layout(
        title_text='Best MLP test accuracy scores for varying layers',
        xaxis_title="hidden layer size",
        yaxis_title="number of hidden layers",
        legend_title="test accuracy",)
    fig.show()

compare_mlp_sizes(df_history_max_test_acc, variant="MLP_SGD")

C:\Users\samlu\AppData\Local\Temp\ipykernel_9080\3193678587.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Description:** Looking the heat map, some interesting things can be observed:

1. It is noticeable that increasing the number of nodes per layer across all layer constellations has led to an improvement in the maximum test accuracy. The best variants based on our test set are those with the maximum tested number of 256 nodes per layer. 


2. Comparing the different numbers of layers, it can be seen that the variants with only one hidden layer performed clearly worse than the variants with more than one hidden layer for all layer sizes (except for layer size 32). The variants between 2 and 5 hidden layers have top test accuracy scores within the same hidden layer size that are much closer to each other.

Based on this result, I would suggest that an MLP on our training data is therefore only partially able to generalise in such a way that it can reliably classify new samples. An extension of layers does not lead to a significant improvement after 2 to 3 layers. 


 #### CNN influence of  layer constellations

In [374]:
def compare_cnn_sizes(df, variant):
    if variant:
        df = df.loc[df["variant"]==variant]
        
    df["conv_layer_size"] = df["conv_layers"].apply(lambda x: x[0])
    df = df[["num_conv", "conv_layer_size", "test accuracy"]].groupby(["num_conv", "conv_layer_size"]).max().reset_index()
    df = df.pivot(index='num_conv', columns='conv_layer_size', values='test accuracy')
   
    fig = go.Figure(data=go.Heatmap(
        z = df.values,
        x = df.columns.astype(str),
        y = df.index.astype(str),
        texttemplate="%{z}",
        textfont={"size":20}
        ))
    
    fig.update_layout(
        title_text='Best CNN test accuracy scores for varying layers',
        xaxis_title="convolutional layer size",
        yaxis_title="number of convolutional layers",
        legend_title="test accuracy",)
    fig.show()


compare_cnn_sizes(df_history_max_test_acc, variant="CNN_SGD")


C:\Users\samlu\AppData\Local\Temp\ipykernel_9080\3598570598.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



As with the MLP, let's look at the best test accuracy for the CNN constalations.

**Description:** When looking at the heat map, we also see an improvement in the best achieved test accuracy with an increasing number of filter kernels per convolutional layer. Thus, the model is able to extract more relevant features from the data set which can be used by the subsequent fully connected layers for the classifiaction task.

It can also be seen that an increase in the number of convolutional layers leads to an improvement. This shows us that our model can generate more complex features with more convolutional layers, which are apparently also relevant for the classification of unseen data.

### Inclusion of Regularization

#### L2 Regularization

The Ridge regression adds the “squared magnitude” of all weights as penalty a term to the loss function. It is relevant how strongly the regularization is weighted in the calculation of the cost function. The strength of the regularization can be set via the hyperparameter *lambda*: $\lambda$. If lambda is 0, then the regularization is not included in the optimization, which corresponds to the OLS variant used before. The higher lambda is chosen, the more the individual weights are decreased. L2 regularization therefore disperse the error terms in all the weights which leads to more accurate customized final models.

In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7

create_sweep = False #create new sweeps


mlp_sweep_configuration = {
    "name": "MLP_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "MLP_SGD_L2"
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":200,
            "min":50
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":180,
            "min":50
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "values": [0, 0.05, 0.1, 0.2]
        },
        "drop_out": {
            "value": 0
        },
    }
}

def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = MLPNet_3_layer(
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7

create_sweep = False #create new sweeps

cnn_sweep_configuration = {
    "name": "CNN_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "CNN_SGD_L2"
        },
        "conv1_channels": {
            "values":[4, 8, 16, 32]
        },
        "conv2_channels": {
            "values":[4, 8, 16, 32]
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":190,
            "min":150
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":190,
            "min":50
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "values": [0, 0.05, 0.1, 0.2]
        },
        "drop_out": {
            "value": 0
        },
        "kernel_size": {
            "value": 5
        },
        "pool_size": {
            "value": 2
        },
    }
}


def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = ConvNet(
            kernel_size=config["kernel_size"],
            pool_size=config["pool_size"],
            conv1_channels=config["conv1_channels"],
            conv2_channels=config["conv2_channels"],
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(cnn_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

**Description:** The effect of L2 regularisation is clearly visible in the training loss. Using the L2 regularisation parameter prevents the training loss from completely converging to 0. The higher the value of the regularisation parameter, the earlier the conversion stops. However, L2 regularisation did not improve the test loss. 

#### Drop out Regulatization

With drop out regularization, a specified percentage of the output values of the nodes within a layer are swept out and their value is set to zero. The selection of the nodes occurs randomly and is repeated for each batch. By ignoring individual node outputs in training, it ensures that the model cannot solely rely on individual nodes for making a classification. This causes the individual edge weights to be lowered overall, which should produce an effect similar to that of L2 regularization.

In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7

create_sweep = False #create new sweeps


mlp_sweep_configuration = {
    "name": "MLP_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "MLP_SGD_DP"
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":200,
            "min":100
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":180,
            "min":70
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "values": [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
        },
    }
}

def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = MLPNet_3_layer(
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7

create_sweep = False #create new sweeps

cnn_sweep_configuration = {
    "name": "CNN_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "CNN_SGD_DP"
        },
        "conv1_channels": {
            "values":[4, 8, 16, 32]
        },
        "conv2_channels": {
            "values":[4, 8, 16, 32]
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":190,
            "min":150
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":190,
            "min":50
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "values": [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
        },
        "kernel_size": {
            "value": 5
        },
        "pool_size": {
            "value": 2
        },
    }
}


def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = ConvNet(
            kernel_size=config["kernel_size"],
            pool_size=config["pool_size"],
            conv1_channels=config["conv1_channels"],
            conv2_channels=config["conv2_channels"],
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(cnn_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

### Including Batchnorm BN (without REG)

Batchnormalization is a popular method to improve the training of deep neural networks, where there is an effect that can occur with an extended number of layers: the so-called *internal covariance shift*. This describes the phenomenon when the distribution of the activation values of a layer is changed by adjusting the weights of the prior layer. The adjustments of all preceding layers thus have an influence on the distributions of all activations of subsequent layers. The subsequent layers therefore must adapt to these changes. As a result, the training is significantly slowed down.

Batch normalisation aims, as the name suggests, to normalise the distribution of activation values per batch for each layer, which reduces the variation of activation values per layer to a more defined range. Batch normalisation can thus also be seen as a kind of smoothing of the cost function in each layer, which in turn reduces the variation of the gradient lengths and thus leads to a more stable learning behaviour.

In [ ]:
class MLPNet_3_layer_bn(nn.Module):
    '''Multi Layer Perceptron Net class with 3 layers'''
    def __init__(self, fc_hidden_1, fc_hidden_2, fc_hidden_3=None, activation=F.relu, drop_out=0):
        super(MLPNet_3_layer_bn, self).__init__()
        self.fc1 = nn.Linear(in_features=32*32*3, out_features=fc_hidden_1)
        self.fc1_bn = nn.BatchNorm1d(fc_hidden_1)
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc2_bn = nn.BatchNorm1d(fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation # activation function
    
    def forward(self, x):
        batch_size  = x.shape[0]
        x = x.view(batch_size, -1)
        x = self.activation(self.fc1_bn(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.fc2_bn(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

class ConvNet_bn(nn.Module):
    def __init__(self, activation=F.relu, kernel_size=5, pool_size=2, conv1_channels = 10, conv2_channels=8, fc_hidden_1=120, fc_hidden_2=84, drop_out=0):
        '''Convolutional Net class'''
        super(ConvNet_bn, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=conv1_channels, kernel_size=kernel_size) 
        self.conv1_bn=nn.BatchNorm2d(conv1_channels)
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
        self.conv2 = nn.Conv2d(in_channels=conv1_channels, out_channels=conv2_channels, kernel_size=kernel_size) 
        self.conv2_bn=nn.BatchNorm2d(conv2_channels)
        self.fc1 = nn.Linear(in_features=conv2_channels*5*5, out_features=fc_hidden_1)
        self.fc1_bn = nn.BatchNorm1d(fc_hidden_1)
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc2_bn = nn.BatchNorm1d(fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation
        self.conv2_channels = conv2_channels


    def forward(self, x):
        x = self.pool(self.activation(self.conv1_bn(self.conv1(x))))
        x = self.pool(self.activation(self.conv2_bn(self.conv2(x))))
        x = x.view(-1, self.conv2_channels*5*5) # flatten tensor
        x = self.activation(self.fc1_bn(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.fc2_bn(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


class ConvNet_3_bn(nn.Module):
    def __init__(self, activation=F.relu, kernel_size=5, pool_size=2, padding=3, conv1_channels = 10, conv2_channels=8, conv3_channels=8, fc_hidden_1=120, fc_hidden_2=84, drop_out=0):
        '''A convolutional neural network with 3 convolutional layers'''
        super(ConvNet_3_bn, self).__init__()   
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=conv1_channels, kernel_size=kernel_size, padding=padding) 
        self.conv1_bn=nn.BatchNorm2d(conv1_channels)
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
        self.conv2 = nn.Conv2d(in_channels=conv1_channels, out_channels=conv2_channels, kernel_size=kernel_size, padding=padding) 
        self.conv2_bn=nn.BatchNorm2d(conv2_channels)
        self.conv3 = nn.Conv2d(in_channels=conv2_channels, out_channels=conv3_channels, kernel_size=kernel_size, padding=padding)
        self.conv3_bn=nn.BatchNorm2d(conv3_channels)
        self.fc1 = nn.Linear(in_features=conv3_channels*5*5, out_features=fc_hidden_1)   
        self.fc1_bn = nn.BatchNorm1d(fc_hidden_1)
        self.fc2 = nn.Linear(in_features=fc_hidden_1, out_features=fc_hidden_2)
        self.fc2_bn = nn.BatchNorm1d(fc_hidden_2)
        self.fc3 = nn.Linear(in_features=fc_hidden_2, out_features=10)
        self.dropout = nn.Dropout(drop_out)
        self.activation = activation
        self.conv3_channels = conv3_channels

    def forward(self, x):
        x = self.pool(self.activation(self.conv1_bn(self.conv1(x))))
        x = self.pool(self.activation(self.conv2_bn(self.conv2(x))))
        x = self.pool(self.activation(self.conv3_bn(self.conv3(x))))
        x = x.view(-1, self.conv3_channels*5*5) # flatten tensor
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7

create_sweep = False #create new sweeps


mlp_sweep_configuration = {
    "name": "MLP_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "MLP_SGD_BN"
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":200,
            "min":100
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":180,
            "min":70
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "value": 0
        },
    }
}

def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = MLPNet_3_layer_bn(
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7

create_sweep = False #create new sweeps

cnn_sweep_configuration = {
    "name": "CNN_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "CNN_SGD_BN"
        },
        "conv1_channels": {
            "values":[4, 8, 16, 32]
        },
        "conv2_channels": {
            "values":[4, 8, 16, 32]
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":190,
            "min":150
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":190,
            "min":50
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "value": 0
        },
        "kernel_size": {
            "value": 5
        },
        "pool_size": {
            "value": 2
        },
    }
}


def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = ConvNet_bn(
            kernel_size=config["kernel_size"],
            pool_size=config["pool_size"],
            conv1_channels=config["conv1_channels"],
            conv2_channels=config["conv2_channels"],
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.SGD(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(cnn_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

### Using Adam (without BN)

In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7
create_sweep = False #create new sweeps


mlp_sweep_configuration = {
    "name": "MLP_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "MLP_SGD_ADAM"
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":200,
            "min":80
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":180,
            "min":50
        },
        "fc_hidden_3": {
            "distribution": "int_uniform",
            "max":180,
            "min":50
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "value": 0
        },
    }
}

def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = MLPNet_4_layer(
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            fc_hidden_3=config["fc_hidden_3"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.Adam(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(mlp_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

In [ ]:
num_epochs = 100
num_runs = 5
min_acc = 0.7
create_sweep = False #create new sweeps

cnn_sweep_configuration = {
    "name": "CNN_sweep",
    "metric": {"name": "test accuracy", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "name": {
            "value": "CNN_SGD_ADAM"
        },
        "conv1_channels": {
            "values":[32, 64, 128]
        },
        "conv2_channels": {
            "values":[32, 64, 128]
        },
        "conv3_channels": {
            "values":[32, 64, 128]
        },
        "fc_hidden_1": {
            "distribution": "int_uniform",
            "max":190,
            "min":150
        },
        "fc_hidden_2": {
            "distribution": "int_uniform",
            "max":190,
            "min":50
        },
        "num_epochs": {
            "value": num_epochs
        },
        "learning_rate": {
            "values": [0.1, 0.05]
        },
        "l2_weight_decay": {
            "value": 0
        },
        "drop_out": {
            "value": 0
        },
        "kernel_size": {
            "value": 5
        },
        "pool_size": {
            "value": 2
        },
    }
}


def train(config=None):
    with wandb.init(entity="simonluder", config=config) as run:
        config = wandb.config
        wandb.run.name = "{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        # define model
        model = ConvNet_3(
            kernel_size=config["kernel_size"],
            pool_size=config["pool_size"],
            conv1_channels=config["conv1_channels"],
            conv2_channels=config["conv2_channels"],
            conv3_channels=config["conv3_channels"],
            fc_hidden_1=config["fc_hidden_1"],
            fc_hidden_2=config["fc_hidden_2"],
            drop_out=config["drop_out"],
            ).to(device) 
        
        # define trainer
        trainer = ModelTrainer(
            model=model, 
            train_loader=train_loader, 
            test_loader=test_loader, 
            num_epochs=config["num_epochs"],
            criterion=nn.CrossEntropyLoss(),
            optimizer=torch.optim.Adam(
                model.parameters(), 
                lr=config["learning_rate"],
                weight_decay = config["l2_weight_decay"],
                )
            )
            
        #train model
        trainer.train(verbose=False, include_test_scores=True)

        # save model if minimal test accuracy-score is reached
        if trainer.best_log["test accuracy"] >= min_acc:
            save_model(path="top_model", model=trainer.best_model, name=wandb.run.name)
       
if create_sweep:
    sweep_id = wandb.sweep(cnn_sweep_configuration, project="del_mc1", entity="simonluder")
    wandb.agent(sweep_id, function=train, count=num_runs)

The first noticeable change is the change in the optimal learning rate. While our previous variants had an optimal learning rate between 0.1 and 0.05, with the Adam optimizer this rate is clearly too high and leads to an unstable learning behaviour. It seems that for our models in the CIFAR10 data set, a 10 times smaller learning rate is necessary to achieve a stable training behaviour.